In [10]:
import warnings
warnings.filterwarnings("ignore")
import json
from lib import config
import numpy as np
import pandas as pd
from lib.country_exposure_summary import Portfolios_Country, Comparison
import datetime as dt


In [11]:
df = pd.read_csv(config.EXTERNAL_DIR.time_series_dir / 'gqg_sector_active_weight.csv',index_col=0)

In [15]:
df2 = pd.read_csv(config.EXTERNAL_DIR.save_dir / 'wasatch_attribution_matrix.csv',index_col=0)

In [20]:
df2['Avg Act Weight'].T

GICS Sector Name
Cash & Others             0.013033
Communication Services   -0.011518
Consumer Discretionary    0.049849
Consumer Staples         -0.020263
Energy                   -0.066611
Financials                0.116083
Health Care              -0.102370
Industrials              -0.046229
Information Technology    0.034024
Materials                 0.053135
Real Estate              -0.001775
Utilities                -0.017358
Name: Avg Act Weight, dtype: float64

In [24]:
df.drop('2023-03-31').append(df2['Avg Act Weight'].rename('2023-03-31')).sort_index()

,Cash & Others,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Date,,,,,,,,,,,,
2023-03-31,0.013033,-0.011518,0.049849,-0.020263,-0.066611,0.116083,-0.102370,-0.046229,0.034024,0.053135,-0.001775,-0.017358
2023-04-30,0.024337,-0.072552,-0.128773,0.084140,0.144032,0.039205,-0.008206,-0.016966,-0.091642,0.014332,-0.019468,0.031562
2023-05-31,0.028120,-0.070304,-0.124973,0.080858,0.139016,0.037517,-0.011322,-0.016621,-0.085784,0.011094,-0.018967,0.031366
2023-06-30,0.028821,-0.066804,-0.124276,0.066199,0.135586,0.034659,-0.011354,-0.016638,-0.069113,0.007470,-0.017104,0.032556
2023-07-31,0.025188,-0.063196,-0.121473,0.060803,0.129824,0.030208,-0.010363,-0.015767,-0.057347,0.002294,-0.015788,0.035616
2023-08-31,0.024407,-0.060505,-0.116764,0.058659,0.126314,0.024284,-0.009979,-0.016557,-0.054134,-0.000594,-0.015138,0.040006
